# Matrikel-Nr: 2235021

# **Painting Classification**

Classify the painter with labeld images of paintings. Implementation with convolutional neural networks & transfer learning.

## **Imports**

In [ ]:
# Matrikel-Nr: 2235021